In [202]:
import requests

import pandas as pd
from bs4 import BeautifulSoup
import spotipy
import spotipy.util as util

# _Import_ Playlists on Beatport to your Spotify  

This Notebook provides functionalities to parse arbitrary playlists on [Beatport](https://www.beatport.com/) ( Beatport is the No. 1 electronic music store for DJs ) and _import_ these playlists into your Spotify account. 

You can see an example for the top playlists of 2018 on Beatport at the bottom. 

## Spotify Authentification 

In [203]:
username = '<spotify-username>' # add your spotify user name
scope = 'playlist-modify-public'

In [204]:
client_id = 'spotify-client-id' # add your spotify client id
client_secret = 'spotify-client-secret' # add your spotify client secrect
redirect_uri = 'http://localhost/'

In [205]:
token = util.prompt_for_user_token(username, scope, client_id, client_secret,redirect_uri)

In [206]:
sp = spotipy.Spotify(auth=token)

## Functions for Parsing Tracks from Beatport Website 

In [112]:
def parse_track(track_html_snippet):
    """Returns Title, Mix and Artist in Dictionary parsed from html snippet of a single track"""
    
    title = track_html_snippet.find(class_="buk-track-primary-title").text
    mix = track_html_snippet.find(class_="buk-track-remixed").text
    artist = track_html_snippet.find(class_="buk-track-artists").find('a').text
    
    return {'title': title, 'mix': mix, 'artist': artist}

In [150]:
def parse_beatport_url(url):
    """parses tracks of beatport staffpicks website
    
    Args:
        url (str): url of beatport website
        
    Returns
        list of dicts containing info about each track, with keys 'title', 'mix', 'artist'
    """
    
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    
    charts_bucket = soup.find(class_='bucket tracks interior-chart-tracks')
    raw_track_list = charts_bucket.find_all(class_='bucket-item')
    
    parsed_track_list = []
    for item in raw_track_list:
        try:
            parsed_track = parse_track(item)
        except BaseException as exc:
            print('####################')
            print(exc)
            print(item)
            print('####################')
        else:
            parsed_track_list.append(parsed_track)
    
    print(f'Parsed {len(parsed_track_list)} tracks')
    
    return parsed_track_list

## Functions for Finding & Creating Playlists in [Spotify](https://spotipy.readthedocs.io/en/latest/#api-reference)

### Find Spotify Titles

In [142]:
def find_track(query_string, sp):
    """returns array with Title, Artist, URI of track that matches query string best
    
    Args:
        query_string (str): query string for spotify search
        sp (spotipy object)
    """
    
    results = sp.search(query_string)
    
    name = results['tracks']['items'][0]['name']
    artist = results['tracks']['items'][0]['artists'][0]['name']
    uri = results['tracks']['items'][0]['uri']
    
    #return [name, artist, uri]
    return uri

In [157]:
def find_tracks(parsed_track_list):
    """Returns list of Spotify Track URIs
    
    Args:
        parsed_track_list (list): list of dicts with keys 'title', 'mix' and 'artist' 
    """
    found_tracks = []
    for track in parsed_track_list:
        found_track = False

        # try to find track with 'track_title mix artist'
        try:
            query = '{} {} {}'.format(track["title"], track["mix"], track["artist"])
            found_track = find_track(query, sp)
            found_tracks.append(found_track)
            continue
        except IndexError:
            pass
            # print(f'{query} not found with mix')

        # if no track was found, find track with 'track_title artist'    
        try:
            query = '{} {}'.format(track["title"], track["artist"])
            found_track = find_track(query, sp) 
            found_tracks.append(found_track)
            # print(f'{query} found without mix')
            continue
        except IndexError:
            pass
            # print(f'{query} not found at all')

    print(f'{len(found_tracks)} tracks found')
    
    return found_tracks

### Create Playlist

In [154]:
def create_playlist(url, name, sp):
    """Creates Spotify Playlist based on Beatport URL
    
    Args:
        url (str): url of beatport staffpicks website
        name (str): name of spotify playlist
        sp (spotipy.client.Spotify): spotify api connection
    """
    
    track_list = parse_beatport_url(url)
    spotify_uris = find_tracks(track_list)
    
    
    playlist_id = sp.user_playlist_create('kaijeggle', name)['id']
    sp.user_playlist_add_tracks('kaijeggle', playlist_id, spotify_uris)
    
    print('Created Spotify Playlist {}'.format(name))
    
    

## Create one Playlist based on Beatport URL

In [155]:
url = 'https://www.beatport.com/chart/staff-picks-2018-techno/541953'
name = 'BEATPORT Techno Staff Picks 2018'

In [156]:
create_playlist(url, name, sp)

Parsed 50 tracks
Energize Original Mix Amelie Lens not found with mix
Energize Amelie Lens found without mix
Your Mind Original Mix Adam Beyer not found with mix
Your Mind Adam Beyer found without mix
No Regrets Original Mix BEC not found with mix
No Regrets BEC found without mix
Outsider Original Mix Joran Van Pol not found with mix
Outsider Joran Van Pol found without mix
Enigma Original Mix Rebekah not found with mix
Enigma Rebekah found without mix
DARE U Original Mix CIREZ D not found with mix
DARE U CIREZ D found without mix
Brussels Original Mix Charlotte de Witte not found with mix
Brussels Charlotte de Witte found without mix
Xingu Original Mix JX-216 not found with mix
Xingu JX-216 found without mix
Prosperity Original Mix Anja Schneider not found with mix
Prosperity Anja Schneider found without mix
Colour of a Dream Original Mix Alan Fitzpatrick not found with mix
Colour of a Dream Alan Fitzpatrick found without mix
Teint Original Mix Amotik not found with mix
Teint Amotik f

## Create Playlist for all [Beatport Staff Picks 2018](https://www.beatport.com/staff-picks-2018)

In [159]:
url = 'https://www.beatport.com/staff-picks-2018'
html = requests.get(url).text
soup = BeautifulSoup(html)

In [184]:
raw_genre_list = soup.find_all(class_='bucket-item ec-item artwork-grid-item charts-artwork-grid-item')

In [192]:
genres = []
for genre in raw_genre_list:
    relative_url = genre.find('a')['href']
    absolute_url = 'https://www.beatport.com' + relative_url
    genre = relative_url.split('2018-')[1].split('/')[0].title()
    
    genres.append({'genre': genre, 'url': absolute_url})

### Create Playlist for every Genre

In [207]:
for genre in genres[-6:]:
    print('Start Process for', genre['genre'])
    playlist_name = f'Beatport {genre["genre"]} Staff Picks 2018'
    create_playlist(genre['url'], playlist_name, sp)

Start Process for Breaks
Parsed 50 tracks
43 tracks found
Created Spotify Playlist Beatport Breaks Staff Picks 2018
Start Process for Dubstep
Parsed 50 tracks
48 tracks found
Created Spotify Playlist Beatport Dubstep Staff Picks 2018
Start Process for Trap-Future-Bass
Parsed 53 tracks
41 tracks found
Created Spotify Playlist Beatport Trap-Future-Bass Staff Picks 2018
Start Process for Hip-Hop
Parsed 51 tracks
48 tracks found
Created Spotify Playlist Beatport Hip-Hop Staff Picks 2018
Start Process for Garage-Bassline-Grime
Parsed 57 tracks
48 tracks found
Created Spotify Playlist Beatport Garage-Bassline-Grime Staff Picks 2018
Start Process for Leftfield-Bass
Parsed 59 tracks
52 tracks found
Created Spotify Playlist Beatport Leftfield-Bass Staff Picks 2018
